In [1]:
import pandas as pd

In [2]:
#
right=pd.read_csv('right_dataset.csv')
left=pd.read_csv('left_dataset.csv')

In [3]:
left.replace(to_replace=pd.NA, value='', inplace=True)

In [4]:
########################################################

In [5]:
class StateDataFrame:

    def __init__(self, left, right):
        self.left = left
        self.right = right
        self.left_states = set(self.left['city'])
        self.right_states = set(self.right['city'])
        self.left_state_dfs = {}
        self.right_state_dfs = {}
        for state in self.left_states:
            self.left_state_dfs[state] = self.left[self.left['city'] == state]
        for state in self.right_states:
            self.right_state_dfs[state] = self.right[self.right['city'] == state]
        
    def get_left_state_df(self, state):
        return self.left_state_dfs[state]
    
    def get_right_state_df(self, state):
        return self.right_state_dfs[state]

In [6]:
city_list_left = list(left['city'].unique())
len(city_list_left)

901

In [7]:
city_list_right= list(right['city'].unique())
len(city_list_right)

155

In [8]:
unique_cities_right = set(city_list_right)
filtered_left = pd.DataFrame(left[left['city'].isin(unique_cities_right)])

In [9]:
unique_cities_left = set(filtered_left['city'].unique())
filtered_right = pd.DataFrame(right[right['city'].isin(unique_cities_left)])

In [10]:
filtered_left['info'] = filtered_left['name'] + ', ' + filtered_left['address']

In [11]:
filtered_right['info'] = filtered_right['name'] + ', ' + filtered_right['address']

In [12]:
#####################################################

In [13]:
# unify zip code 
import string as st
filtered_left = filtered_left.dropna()
filtered_right = filtered_right.dropna()

filtered_left['postal_code'] = [int(float(i))  for i in filtered_left['postal_code']] 
filtered_right['zip_code'] = [int(i[:5])  for i in filtered_right['zip_code']] 

In [14]:
# zip code intersection to filter non matching cases
shared_zipcodes = [v for v in filtered_left['postal_code'] if v in filtered_right['zip_code']]
filtered_left = filtered_left[filtered_left['postal_code'].isin(shared_zipcodes)]
filtered_right = filtered_right[filtered_right['zip_code'].isin(shared_zipcodes)]

In [15]:
# clean info 
filtered_left['info'] = [i.lower().translate(str.maketrans('', '', st.punctuation)) for i in filtered_left['info']]
filtered_right['info'] = [i.lower().translate(str.maketrans('', '', st.punctuation)) for i in filtered_right['info']]

In [16]:
# try tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# merge all info
info_all = pd.concat([filtered_left['info'], filtered_right['info']])

# tfidf then dot product 
tfidf = TfidfVectorizer().fit_transform(info_all)
similarity = tfidf[:len(filtered_left)] * tfidf[len(filtered_left):].T

# filter for high similarity score and mark location 

similarity_columns = []
for i in range(len(filtered_left)):
    list1 = [i,np.flatnonzero(similarity[i].toarray() > 0.8)]
    similarity_columns.append(list1) 

# adding all the information 
matches = []
for i in range(len(filtered_left)):
    for v in similarity_columns[i][1].T:
        matches.append([filtered_left.iloc[i, 0],
                       filtered_right.iloc[v, 0],
                       similarity[i,v]* 100])

In [23]:
len(matches)

7022